In [1]:
import pandas as pd
import numpy as np

In [27]:
dataset = pd.read_csv('pred_lstm18.csv')
dataset_2019 = pd.read_csv('pred_lstm19.csv')
dist = pd.read_csv('dataset/Distance_Matrix.csv')
dist.drop(columns="Unnamed: 0")

0         1         2         3         4         5         6  \
0       0.0000   11.3769   20.4557   38.1227   45.3810   54.9915   78.6108   
1      11.3769    0.0000    9.0788   28.9141   36.1724   45.7829   69.4022   
2      20.4557    9.0788    0.0000   22.3791   29.6374   39.2478   62.8671   
3      38.1227   28.9141   22.3791    0.0000   11.8343   23.5413   41.8396   
4      45.3810   36.1724   29.6374   11.8343    0.0000   11.7070   24.3986   
...        ...       ...       ...       ...       ...       ...       ...   
2413  671.2005  668.9528  669.5556  669.9084  652.1102  640.4032  627.7116   
2414  663.4455  661.1978  661.8006  662.1534  644.3551  632.6481  619.9566   
2415  663.7748  661.5271  662.1298  662.4826  644.6844  632.9774  620.2859   
2416  671.0165  668.7688  669.3715  669.7244  651.9261  640.2191  627.5276   
2417  679.2328  676.9851  677.5878  677.9406  660.1424  648.4354  635.7439   

             7         8         9  ...      2408      2409      2410  \
0     118.6750  102.6639  113.4309  ...  683.8771  687.6310  697.3246   
1      78.2329   93.4553  111.1832  ...  681.6295  685.3833  695.0769   
2      71.6979   86.9203  111.7859  ...  682.2323  685.9861  695.6796   
3      50.6703   65.8927   82.5852  ...  681.4226  685.1765  694.8701   
4      33.2293   53.9901   65.1442  ...  663.9816  667.7355  677.4291   
...        ...       ...       ...  ...       ...       ...       ...   
2413  622.9663  638.9532  617.7221  ...  109.0007  112.7545  122.4481   
2414  615.2112  631.1981  609.9671  ...  101.2456  104.9995  114.6931   
2415  615.5405  631.5274  610.2964  ...   84.1469   87.9008   97.5943   
2416  622.7822  638.7691  617.5381  ...   91.3886   95.1425  104.8361   
2417  630.9985  646.9854  625.7544  ...   99.6049  103.3587  113.0523   

          2411      2412      2413      2414      2415      2416      2417  
0     669.3962  667.6788  665.5775  662.0291  665.9655  673.2073  681.4235  
1     667.1485  665.4311  663.3298  659.7815  663.7178  670.9596  679.1758  
2     667.7513  666.0339  663.9326  660.3843  664.3206  671.5623  679.7786  
3     666.9417  665.2243  663.1230  659.5746  663.5110  670.7528  678.9690  
4     649.5007  647.7833  645.6820  642.1336  646.0700  653.3118  661.5280  
...        ...       ...       ...       ...       ...       ...       ...  
2413   12.3091    9.1558    0.0000   14.5629   22.3860   36.6284   44.8446  
2414   26.4955   23.3422   14.1864    0.0000   14.5984   28.8407   37.0570  
2415   34.3098   31.1565   22.0007   12.8105    0.0000   14.2423   22.4586  
2416   48.5521   45.3988   36.2430   27.0528   14.2423    0.0000   12.4741  
2417   56.7684   53.6151   44.4593   35.2691   22.4586   12.4741    0.0000  

[2418 rows x 2418 columns]

In [3]:
arr = [i[0] for i in dataset.values]
arr1 = [i[0] for i in dataset_2019.values]
arr2 = [min(arr[i], arr1[i]) for i in range(len(arr))]

print(sum(arr1), sum(arr), sum(arr2))

336415.76609212795 348012.75765273994 284027.8982988337


In [28]:
sad = [list(i) for i in list(dist.values)]

In [29]:
min(arr)

0.53026325

In [49]:
from pulp import *
import random as rnd

sites = [f"Site_{i}" for i in range(300)]
depots = [f"{i}" for i in range(300)]

supply = {f"Site_{i}":arr[i] for i in range(300)}
supply_2019 = {f"Site_{i}":arr1[i] for i in range(300)}

refineries = [f"{i}" for i in range(300)]

demand = {f"{i}":19999.99 for i in range(300)}
demand_ref = {f"{i}":99999.99 for i in range(300)}

sad = [list(i) for i in list(dist.values)]
distance = sad
distance = makeDict([sites, depots], distance, 0)

prob = LpProblem("Depot_Problem", LpMinimize)

Routes = [(w, b) for w in sites for b in depots]

depot_var = LpVariable.dicts("Supply_to_depots",depots, 0, 1, LpInteger)

vars = LpVariable.dicts("Route", (sites, depots), 0, None, LpInteger)

# Ref
ref_vars = LpVariable.dicts("Route_ref", (depots, refineries), 0, None, LpInteger)
ref_var = LpVariable.dicts("Supply_to_refineries",refineries, 0, 1, LpInteger)
Routes_ref = [(w, b) for w in depots for b in refineries]

main_obj = (lpSum(vars[w][b] * distance[w][b] for (w, b) in Routes)*0.001)
ref_constraint = (lpSum(ref_vars[b][k] * distance[b][k] for (b, k) in Routes_ref)*0.001)

prob += ref_constraint + main_obj

for w in sites:
    prob += (
        lpSum(vars[w][b] for b in depots) <= supply[w],
        f"Sites must not give more than they produce{w}"
    )

#Refineries
for b in depots:
    prob += (
        lpSum(ref_vars[b][k] for k in refineries) <= (lpSum(vars[w][b] for w in sites)),
        f"Depot must not give more than they produce{b}"
    )

for b in depots:
    prob += (
        lpSum([vars[w][b] for w in sites]) <= demand[b]*depot_var[b],
        f"Sum of products into depot should be less than its capacity {b}"
    )

#Refineries
for b in refineries:
    prob += (
        lpSum([ref_vars[j][b] for j in depots]) <= demand_ref[b]*ref_var[b],
        f"Sum of products into refineries should be less than its capacity {b}"
    )

prob += lpSum(depot_var[b] for b in depots) <= max(1, min(25,(0.8*sum(supply.values())//20000)+1))
prob += lpSum(depot_var[b] for b in depots) <= max(1, min(25,(0.8*sum(supply_2019.values())//20000)+1))
prob += lpSum(ref_var[b] for b in refineries) <= max(1, min(5,(0.8*sum(supply.values())//100000)+1))
prob += lpSum(ref_var[b] for b in refineries) <= max(1, min(5,(0.8*sum(supply_2019.values())//100000)+1))

huf = ()
for w in sites:
    for b in depots:
        if depot_var[b] >= 1:
            huf += (
            lpSum(vars[w][b] )
            )

prob += lpSum(huf) >= sum(supply.values())*0.8
prob += lpSum(huf) >= sum(supply_2019.values())*0.8

# Contraint 8: Amount entering into depot equals amount exiting depot
for b in depots:
    if depot_var[b] >= 1:
        prob += (
            lpSum(vars[w][b] for w in sites) <= lpSum(ref_vars[b][k] for k in refineries if ref_var[k] >= 1)
        )
    
# Refineries
hufe = ()
for w in depots:
    for b in refineries:
        if ref_var[b] >= 1:
            hufe += (
            lpSum(ref_vars[w][b] )
            )

prob += lpSum(hufe) >= sum(supply.values())*0.8
prob += lpSum(hufe) >= sum(supply_2019.values())*0.8

solver_options = {'msg': True, 'timeLimit': 1500}  # Set solver-specific options
prob.solve(CPLEX_CMD(**solver_options))

# for i in vars:
#     for each in vars[i]:
#         if vars[i][each].varValue > 0:
#             print(f"TP {vars[i][each]} ---  {vars[i][each].varValue}")
# print("\nSupply ref ")
# for i in ref_vars:
#     for each in ref_vars[i]:
#         if ref_vars[i][each].varValue > 0:
#             print(f"TP {ref_vars[i][each]} ---  {ref_vars[i][each].varValue}")
            
print("Supplied Depots:")
for b in depots:
    if depot_var[b].varValue == 1:
        print(b)

print("\nSupplied Refineries:")
for b in refineries:
    if ref_var[b].varValue == 1:
        print(b)

print(f"\n{sum(supply.values())}")
print((sum(supply.values())//20000)+1)
print(sum([(lpSum(vars[w][b].varValue for w in sites)) for b in depots if depot_var[b].varValue >= 1]))
print(sum([(lpSum(ref_vars[w][b].varValue for w in depots)) for b in refineries if ref_var[b].varValue >= 1]))
# print([depot_var[i].varValue for i in depot_var if depot_var[i] == 1])
# print(supply.values())

KeyboardInterrupt: 

In [7]:
([(lpSum(vars[w][b].varValue for w in sites)) for b in depots if depot_var[b].varValue >= 1])

[12562.0, 14550.0, 16100.0]

In [46]:
dist.values[0][1]

0.0

In [48]:
for i in vars:
    for each in vars[i]:
        if vars[i][each].varValue > 0:
            print(f"TP {vars[i][each]} ---  {vars[i][each].varValue}")

TP Route_Site_0_0 ---  8.0
TP Route_Site_1_0 ---  29.0
TP Route_Site_2_1 ---  62.0
TP Route_Site_3_1 ---  80.0
TP Route_Site_4_1 ---  20.0
TP Route_Site_5_1 ---  31.0
TP Route_Site_6_1 ---  38.0
TP Route_Site_10_0 ---  16.0
TP Route_Site_11_0 ---  3.0
TP Route_Site_12_0 ---  2.0
TP Route_Site_14_0 ---  1.0
TP Route_Site_15_0 ---  63.0
TP Route_Site_16_0 ---  85.0
TP Route_Site_17_0 ---  159.0
TP Route_Site_18_0 ---  135.0
TP Route_Site_19_0 ---  90.0
TP Route_Site_20_0 ---  116.0
TP Route_Site_21_0 ---  118.0
TP Route_Site_22_0 ---  148.0
TP Route_Site_23_0 ---  172.0
TP Route_Site_24_0 ---  171.0
TP Route_Site_25_0 ---  68.0
TP Route_Site_26_0 ---  56.0
TP Route_Site_27_1 ---  40.0
TP Route_Site_28_1 ---  6.0
TP Route_Site_30_1 ---  35.0
TP Route_Site_31_1 ---  130.0
TP Route_Site_32_1 ---  170.0
TP Route_Site_33_1 ---  124.0
TP Route_Site_34_1 ---  153.0
TP Route_Site_35_1 ---  165.0
TP Route_Site_36_1 ---  156.0
TP Route_Site_37_1 ---  172.0
TP Route_Site_38_1 ---  98.0
TP Route_Sit

In [ ]:
from pulp import *
import random as rnd

sites = [f"Site_{i}" for i in range(100)]
depots = [f"{i}" for i in range(100)]

supply = {f"Site_{i}":arr[i] for i in range(100)}
supply_2019 = {f"Site_{i}":arr[i] for i in range(100)}

refineries = [f"{i}" for i in range(100)]

demand = {f"{i}":19999.99 for i in range(100)}
demand_ref = {f"{i}":99999.99 for i in range(100)}

sad = [list(i) for i in list(dist.values)]
distance = sad
distance = makeDict([sites, depots], distance, 0)

prob = LpProblem("Depot_Problem", LpMinimize)

Routes = [(w, b) for w in sites for b in depots]

depot_var = LpVariable.dicts("Supply_to_depots",depots, 0, 1, LpInteger)

vars = LpVariable.dicts("Route", (sites, depots), 0, None, LpInteger)

# Ref
ref_vars = LpVariable.dicts("Route_ref", (depots, refineries), 0, None, LpInteger)
ref_var = LpVariable.dicts("Supply_to_refineries",refineries, 0, 1, LpInteger)
Routes_ref = [(w, b) for w in depots for b in refineries]

main_obj = (lpSum(vars[w][b] * distance[w][b] for (w, b) in Routes)*0.001)
ref_constraint = lpSum(ref_vars[b][k] * distance[b][k] for (b, k) in Routes_ref)*0.001

prob += ref_constraint + main_obj

for w in sites:
    prob += (
        lpSum(vars[w][b] for b in depots) <= supply[w],
        f"Sites must not give more than they produce{w}"
    )

#Refineries
for b in depots:
    prob += (
        lpSum(ref_vars[b][k] for k in refineries) <= (lpSum(vars[w][b] for w in sites)),
        f"Depot must not give more than they produce{b}"
    )

for b in depots:
    prob += (
        lpSum([vars[w][b] for w in sites]) <= demand[b]*depot_var[b],
        f"Sum of products into depot should be less than its capacity {b}"
    )

#Refineries
for b in refineries:
    prob += (
        lpSum([ref_vars[j][b] for j in depots]) <= demand_ref[b]*ref_var[b],
        f"Sum of products into refineries should be less than its capacity {b}"
    )

prob += lpSum(depot_var[b] for b in depots) <= max(1, min(25,(0.8*sum(supply.values())//20000)+1))
prob += lpSum(ref_var[b] for b in refineries) <= max(1, min(5,(0.8*sum(supply.values())//100000)+1))

huf = ()
for w in sites:
    for b in depots:
        if depot_var[b] >= 1:
            huf += (
            lpSum(vars[w][b] )
            )

prob += lpSum(huf) >= sum(supply.values())*0.8

# Contraint 8: Amount entering into depot equals amount exiting depot
for b in depots:
    if depot_var[b] >= 1:
        prob += (
            lpSum(vars[w][b] for w in sites) <= lpSum(ref_vars[b][k] for k in refineries if ref_var[k] >= 1)
        )
    
# Refineries
hufe = ()
for w in depots:
    for b in refineries:
        if ref_var[b] >= 1:
            hufe += (
            lpSum(ref_vars[w][b] )
            )

prob += lpSum(hufe) >= sum(supply.values())*0.8

solver_options = {'msg': True, 'timeLimit': 1500}  # Set solver-specific options
prob.solve(CPLEX_CMD(**solver_options))

# for i in vars:
#     for each in vars[i]:
#         if vars[i][each].varValue > 0:
#             print(f"TP {vars[i][each]} ---  {vars[i][each].varValue}")
# print("\nSupply ref ")
# for i in ref_vars:
#     for each in ref_vars[i]:
#         if ref_vars[i][each].varValue > 0:
#             print(f"TP {ref_vars[i][each]} ---  {ref_vars[i][each].varValue}")
            
print("Supplied Depots:")
for b in depots:
    if depot_var[b].varValue == 1:
        print(b)

print("\nSupplied Refineries:")
for b in refineries:
    if ref_var[b].varValue == 1:
        print(b)

print(f"\n{sum(supply.values())}")
print((sum(supply.values())//20000)+1)
print(sum([(lpSum(vars[w][b].varValue for w in sites)) for b in depots if depot_var[b].varValue >= 1]))
print(sum([(lpSum(ref_vars[w][b].varValue for w in depots)) for b in refineries if ref_var[b].varValue >= 1]))
# print([depot_var[i].varValue for i in depot_var if depot_var[i] == 1])
# print(supply.values())